# Neural Collaborative Filtering vs Matrix Factorization Revisited


https://arxiv.org/pdf/2005.09683.pdf

추천시스템의 방법론 중에 Neural Net을 활용한 Neural Collaborative Filtering(NeuMF)가 전통적인 Matrix Factorization(MF)를 대체 방안이 될 수 있는지를 다양한 선례로 알아보는 Article

## Abstract
전통적으로 dot product나 비슷한 수준을 상위로 정렬하는 방식은 추천시스템에 많이 채용되었다. 그러나 최근에는 이러한 유사성을 Neural Net으로 학습하는 Neural Collaborative Filitering 방식이 등장하고 있다. 해당 Article에서는 NeuMF를 위한 적절한 파라미터, dot product를 학습하기 위한 MLP의 추정함수 그리고 MLP 기반 알고리즘들이 너무 고비용적이고 왜 dot product 기반의 알고리즘을 사용해야 하는지 알아보자. 마지막으로 제목의 질문이 DNN을 기반으로 하는 NeuMF가 더 좋다라는 답변을 내포하지 않는 이유를 살펴본다.

## Prequisite
Neural Collaborative Filtering 

https://arxiv.org/pdf/1708.05031.pdf

## Definitions
$X: Matrix; x: vectors; x: scalar$<br/>
$[x,z]: x와 z벡터의 concatenation$<br/>
$\phi: R^d \times R^d\rightarrow R: 두개의 n차원 임베딩 벡터의 결합\ p\in R^d and\ q \in R^d$<br/>
p는 user embedding, q는 item embedding<br/>
$\phi(p,q): user와\ item의\ 유사도$

<div align="center">
<img src="imgs/NeuMF_vs_MF_simliarity.jpg" />
</div>

### Dot Product

$\phi^dot(p,q):=<p,q>=p^Tq=\overset{n}{\underset{f=1}{\sum}}p_fq_f$

### Learned Similarity

* $f_{w,b}(x) = \sigma(Wx+b), \sigma(z)=[\sigma(z_1),...,\sigma(z_{out})]$<br/>
$f:R^{d_{in}}\rightarrow R^{d_{out}};\ W\in R^{in\times out};\ b\in R^{out}; \sigma: R\rightarrow R$<br/>
* $\phi^{MLP}(p,q):=f_{W_l,b_l}(...f_{W_1,b_1}([p,q])...)$<br/>
* $\phi^{NeuMF}(p,q):=\phi^{MLP}(p_{[1,...,j],q_{[i...j]}}) + \phi^{GMF}(p_{[1,...,j],q_{[i...j]}})$<br/>
* $\phi^{GMF}(p,q):=\sigma(w_T(p\odot{q}))=\sigma(<w\odot{p,q}>)=\sigma(\overset{d}{\underset{f=1}{\sum}}w_fp_fq_f)$
